In [ ]:
import kwant
from math import sin, cos
import matplotlib.pyplot as plt
import numpy as np

import scipy.constants
import cmath

import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import sns_system
import supercurrent_matsubara
import supercurrent

In [ ]:
syst_pars = dict(
    L_m = 400,
    L_x = 800,
    L_sc_up = 20,
    L_sc_down = 20,
    z_x = 400,
    z_y = 125,
    a   = 10,
    parallel_curve = True,
    transverse_soi = True,
    mu_from_bottom_of_spin_orbit_bands = True,
    k_x_in_sc = True,
    wraparound = False,
    current = False,
    ns_junction=True)

In [ ]:
%%time
syst, site_colors, hopping = sns_system.make_system(**syst_pars)

In [ ]:
coloring =  {'middle_interior' : 'grey',
             'middle_barrier' : 'black',
             'bottom_superconductor' : 'gold',
             'top_superconductor' : 'gold',
             'top_cut' : 'red',
             'bottom_cut' : 'blue'}

for k,v in site_colors.items():
    if site_colors.get(k) is not None:
        site_colors[k] = coloring[v]

In [ ]:
kwant.plot(syst,site_color=sns_system.site_color_function(site_colors, syst), dpi=400);

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = .3,
                 phase = np.pi,
                 T = 0.0,
                 V = 10)

params = dict(**constants,
              **params_raw)